In [1]:
import sqlalchemy
from generate_df import *
import real_inf
from sqlalchemy import create_engine
import pymysql
import pandas as pd

In [2]:
#cria município
print("Lendo arquivo de municípios")
arquivo = 'c3d64a3788342bbdd97d01ef7694f1a0.xlsx'
municipio = Municipio("Município")
municipio.fill_table(arquivo)
municipio.create_df()
df_municipio = municipio.out_df()
# removendo último digito do código do municipio
df_municipio = df_municipio.astype({"Código" : str})
df_municipio['Código'] = df_municipio['Código'].str[:-3]
# ---------------------------------------------------------
df_municipio = df_municipio.drop(df_municipio.loc[645:659].index).astype({"Código" : int}) # para remover o rodapé como "notas" e converter o código para inteiro

Lendo arquivo de municípios


In [3]:
#cria o resto daqui pra baixo
print("Lendo arquivo com outros dados")
arquivo_2 = 'part-00000-9cca567b-94bf-4d83-88ae-c2efe6fb4794.c000.csv'
df_other_info = other_info(arquivo_2)
df_other_info = df_other_info.drop(['vacina_fabricante_referencia', 'paciente_endereco_cep'], axis=1) # removendo colunas com muitos vazios
df_other_info = df_other_info.loc[df_other_info['vacina_categoria_codigo'].notnull()].astype({"vacina_categoria_codigo" : str})
df_other_info['vacina_categoria_codigo'] = df_other_info['vacina_categoria_codigo'].str[:-2]
df_other_info = df_other_info.astype({"vacina_categoria_codigo" : int})
df_other_info = df_other_info.loc[df_other_info['paciente_endereco_coIbgeMunicipio'].notnull()].astype({"paciente_endereco_coIbgeMunicipio" : int}) # converte o código de cidade para inteiro
df_other_info = df_other_info.astype({"vacina_categoria_codigo" : int})
df_other_info['vacina_dataAplicacao'] = df_other_info['vacina_dataAplicacao'].str[:-14]

Lendo arquivo com outros dados


In [4]:
# GENERATING IDS

dose_id = fake_id("dose",10,len(df_other_info["vacina_lote"]))
lab_id = fake_id("lab",5,len(df_other_info["vacina_fabricante_nome"].unique().tolist()))
lab_vac_id = fake_id("lab",5,len(df_other_info["vacina_fabricante_nome"].tolist()))

In [32]:
# cria pessoa
print("Gerando DataFrame Pessoa")
pessoa = Pessoa("Pessoa")
pessoa.fill_table(df_other_info)
pessoa.create_df()
df_pessoa = pessoa.out_df()
df_pessoa = df_pessoa.drop_duplicates(keep='first', subset=['Id'])
#df_pessoa.loc[:,'Data_de_Nascimento'] = pd.to_datetime(df_pessoa['Data_de_Nascimento'], format='%Y-%m-%d')

Gerando DataFrame Pessoa


In [21]:
#cria vacina
print("Gerando DataFrame Vacina")
vacina = Vacina("Vacina")
vacina.fill_table(df_other_info)
vacina.create_df()
df_vacina = vacina.out_df()
df_vacina = df_vacina.drop_duplicates(keep='first', subset=['IdVacina'])

Gerando DataFrame Vacina


In [5]:
#cria laboratório
print("Gerando DataFrame Laboratorio")
laboratorio = Laboratorio("Laboratorio")
laboratorio.fill_table(df_other_info,lab_id)
laboratorio.create_df()
df_laboratorio = laboratorio.out_df()

Gerando DataFrame Laboratorio


In [28]:
#cria dose
print("Gerando DataFrame Dose")
dose = Dose("Dose")
dose.fill_table(df_other_info,dose_id)
dose.create_df()
df_dose = dose.out_df()
df_dose.loc[:, 'IdDose'] = df_dose['IdDose'].str.lower()
df_dose = df_dose.drop_duplicates(keep='first', subset=['IdDose'])

Gerando DataFrame Dose


In [29]:
#cria Unidade Saúde
print("Gerando DataFrame Unidade Saude")
unidade_saude = Unidade_Saude("Unidade_Saude")
unidade_saude.fill_table(df_other_info)
unidade_saude.create_df()
df_unidade_saude = unidade_saude.out_df()
df_unidade_saude = df_unidade_saude.drop_duplicates(keep='first', subset=['IdUBS'])

Gerando DataFrame Unidade Saude


In [30]:
#cria aplica em
print("Gerando DataFrame Aplica_Em")
aplicada_em = Aplicada_Em("Aplica_Em")
aplicada_em.fill_table(df_other_info,dose_id)
aplicada_em.create_df()
df_aplicada_em = aplicada_em.out_df()
df_aplicada_em.loc[:, 'IdDose'] = df_aplicada_em['IdDose'].str.lower()
df_aplicada_em = df_aplicada_em.drop_duplicates(keep='first', subset=['IdPessoa', 'IdDose'])

Gerando DataFrame Aplica_Em


AttributeError: 'DataFrame' object has no attribute 'drop_duplicate'

In [34]:
#cria habita em
print("Gerando DataFrame Habita_Em")
habita_em = Habita_Em("Habita_Em")
habita_em.fill_table(df_other_info)
habita_em.create_df()
df_habita_em = habita_em.out_df()
df_habita_em = df_habita_em.drop_duplicates(keep='first')

Gerando DataFrame Habita_Em


AttributeError: 'DataFrame' object has no attribute 'drop_duplicate'

In [36]:
#cria tem
print("Gerando DataFrame Tem")
tem = Tem("Tem")
tem.fill_table(df_other_info,dose_id)
tem.create_df()
df_tem = tem.out_df()
df_tem = df_tem.drop_duplicates(keep='first')

Gerando DataFrame Tem


In [37]:
#cria fica no
print("Gerando DataFrame Fica_No")
fica_no = Fica_No("Fica_No")
fica_no.fill_table(df_other_info)
fica_no.create_df()
df_fica_no = fica_no.out_df()
df_fica_no = df_fica_no.drop_duplicates(keep='first')

Gerando DataFrame Fica_No


In [38]:
#cria enviada para
print("Gerando DataFrame Enviada_Para")
enviada_para = Enviada_Para("Enviada_Para")
enviada_para.fill_table(df_other_info)
enviada_para.create_df()
df_enviada_para = enviada_para.out_df()
df_enviada_para = df_enviada_para.drop_duplicates(keep='first')

Gerando DataFrame Enviada_Para


In [39]:
#cria do tipo
print("Gerando DataFrame Do_Tipo")
do_tipo = Do_Tipo("Do_Tipo")
do_tipo.fill_table(df_other_info, dose_id)
do_tipo.create_df()
df_do_tipo = do_tipo.out_df()
df_do_tipo.loc[:, 'IdDose'] = df_do_tipo['IdDose'].str.lower()
df_do_tipo = df_do_tipo.drop_duplicates(keep='first')

Gerando DataFrame Do_Tipo


In [40]:
#cria porduzida por
print("Gerando DataFrame Produzida_Por")
produzida_por = Produzida_Por("Produzida_Por")
produzida_por.fill_table(df_other_info, lab_vac_id)
produzida_por.create_df()
df_produzida_por = produzida_por.out_df()
df_produzida_por = df_produzida_por.drop_duplicates(keep='first')

Gerando DataFrame Produzida_Por


In [2]:
# Credentials to database connection ADD
hostname="localhost"
dbname="teste2"
username="enzo"
pwd="password"

connection = pymysql.connect(
    host=hostname,
    user=username,
    password=pwd,
    db=dbname
    )
cursor = connection.cursor()

# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=username, pw=pwd))

In [33]:
# Convert dataframe to sql table
print("Criando tabela Pessoa")
df_pessoa.to_sql('pessoas', engine, index=False,
            dtype={
                'Id' : sqlalchemy.types.VARCHAR(length=64),
                'CPF' : sqlalchemy.types.BIGINT,
                'Nome' : sqlalchemy.types.VARCHAR(length=50),
                'Gênero' : sqlalchemy.types.VARCHAR(length=1),
                'Idade' : sqlalchemy.types.SMALLINT,
                'Data_de_Nascimento' : sqlalchemy.types.DATE,
                'Etnia_código' : sqlalchemy.types.SMALLINT,
                'Etnia' : sqlalchemy.types.VARCHAR(length=15),
                'Ocupação' : sqlalchemy.types.VARCHAR(length=50),
                'Grupo' : sqlalchemy.types.VARCHAR(length=50)

            })
print("Tabela Pessoa criada")

Criando tabela Pessoa
Tabela Pessoa criada


In [41]:
print("Criando tabela Município")
df_municipio.to_sql('municipios', engine, index=False,
                   dtype={
                     'Código' : sqlalchemy.types.INT,
                     'População' : sqlalchemy.types.INT,
                     'Nome' : sqlalchemy.types.VARCHAR(length=30),
                   })
print("Tabela Pessoa criada")

Criando tabela Município
Tabela Pessoa criada


In [42]:
print("Criando tabela Vacinas")
df_vacina.to_sql('vacinas', engine, index=False,
                dtype={
                    'IdVacina' : sqlalchemy.types.SMALLINT,
                    'Nome' : sqlalchemy.types.VARCHAR(length=64)
                })
print("Tabela Vacinas criada")

Criando tabela Vacinas
Tabela Vacinas criada


In [43]:
print("Criando tabela Dose")
df_dose.to_sql('doses', engine, index=False,
              dtype={
                  'IdDose' : sqlalchemy.types.VARCHAR(length=15),
                  'Data de Validade' : sqlalchemy.types.DATE,
                  'Número' : sqlalchemy.types.VARCHAR(length=11)
              })
print("Tabela Doses criada")

Criando tabela Dose
Tabela Doses criada


In [44]:
print("Criando tabela Unidade Saude")
df_unidade_saude.to_sql('Unidade Saude', engine, index=False,
                       dtype={
                           'IdUBS' : sqlalchemy.types.INT,
                           'Nome' : sqlalchemy.types.VARCHAR(length=60),
                           'Endereco' : sqlalchemy.types.VARCHAR(length=100)
                       })
print("Tabela Unidade Saude criada")

Criando tabela Unidade Saude
Tabela Unidade Saude criada


c:\users\enzoc\appdata\local\programs\python\python39\lib\site-packages\pandas\io\sql.py:1422: UserWarning: The provided table name 'Unidade Saude' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [45]:
print("Criando tabela Laboratorio")
df_laboratorio.to_sql('laboratorio', engine, index=False,
                     dtype={
                         'IdLaboratorio' : sqlalchemy.types.VARCHAR(length=10),
                         'Pais' : sqlalchemy.types.VARCHAR(length=30),
                         'Nome' : sqlalchemy.types.VARCHAR(length=32)
                     })
print("Tabela Laboratorio criada")

Criando tabela Laboratorio
Tabela Laboratorio criada


In [46]:
print("Criando tabela Aplicada_Em")
df_aplicada_em.to_sql('Aplicada_em', engine, index=False,
                     dtype={
                         'IdPessoa' : sqlalchemy.types.VARCHAR(length=64),
                         'IdDose' : sqlalchemy.types.VARCHAR(length=15),
                         'Data' : sqlalchemy.types.DATE
                     })
print("Tabela Aplicada_Em criada")

Criando tabela Aplicada_Em
Tabela Aplicada_Em criada


c:\users\enzoc\appdata\local\programs\python\python39\lib\site-packages\pandas\io\sql.py:1422: UserWarning: The provided table name 'Aplicada_em' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [47]:
print("Criando tabela Habita_Em")
df_habita_em.to_sql('Habita_Em', engine, index=False,
                   dtype={
                       'IdPessoa' : sqlalchemy.types.VARCHAR(length=64),
                       'IdMunicipio' : sqlalchemy.types.INT
                   })
print("Tabela Habita_Em criada")

Criando tabela Habita_Em
Tabela Habita_Em criada


c:\users\enzoc\appdata\local\programs\python\python39\lib\site-packages\pandas\io\sql.py:1422: UserWarning: The provided table name 'Habita_Em' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [48]:
print("Criando tabela Tem")
df_tem.to_sql('Tem', engine, index=False,
             dtype={
                 'IdDose' : sqlalchemy.types.VARCHAR(length=15),
                 'IdUBS' : sqlalchemy.types.INT
             })
print("Tabela Tem criada")

Criando tabela Tem
Tabela Tem criada


c:\users\enzoc\appdata\local\programs\python\python39\lib\site-packages\pandas\io\sql.py:1422: UserWarning: The provided table name 'Tem' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [49]:
print("Criando tabela Fica_No")
df_fica_no.to_sql('Fica_No', engine, index=False,
                 dtype={
                     'IdUBS' : sqlalchemy.types.INT,
                     'IdMunicipio' : sqlalchemy.types.INT
                 })
print("Tabela Fica_No criada")

Criando tabela Fica_No
Tabela Fica_No criada


c:\users\enzoc\appdata\local\programs\python\python39\lib\site-packages\pandas\io\sql.py:1422: UserWarning: The provided table name 'Fica_No' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [50]:
print("Criando tabela Enviada_Para")
df_enviada_para.to_sql('Enviada_Para', engine, index=False,
                      dtype={
                          'IdVacina' : sqlalchemy.types.SMALLINT,
                          'IdUBS' : sqlalchemy.types.INT
                      })
print("Tabela Enviada_Para criada")

Criando tabela Enviada_Para
Tabela Enviada_Para criada


c:\users\enzoc\appdata\local\programs\python\python39\lib\site-packages\pandas\io\sql.py:1422: UserWarning: The provided table name 'Enviada_Para' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [51]:
print("Criando tabela Do_Tipo")
df_do_tipo.to_sql('Do_Tipo', engine, index=False,
                 dtype={
                     'IdDose' : sqlalchemy.types.VARCHAR(length=15),
                     'IdVacina' : sqlalchemy.types.SMALLINT
                 })
print("Tabela Do_Tipo criada")

Criando tabela Do_Tipo
Tabela Do_Tipo criada


c:\users\enzoc\appdata\local\programs\python\python39\lib\site-packages\pandas\io\sql.py:1422: UserWarning: The provided table name 'Do_Tipo' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [52]:
print("Criando tabela Produzida_Por")
df_produzida_por.to_sql('Produzida_Por', engine, index=False,
                       dtype={
                           'IdVacina' : sqlalchemy.types.SMALLINT,
                           'IdLaboratorio' : sqlalchemy.types.VARCHAR(length=10)
                       })
print("Produzida_Por criada")

Criando tabela Produzida_Por
Produzida_Por criada


c:\users\enzoc\appdata\local\programs\python\python39\lib\site-packages\pandas\io\sql.py:1422: UserWarning: The provided table name 'Produzida_Por' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [53]:
# Definindo primary key
print('Criando PK de doses')
cursor.execute("ALTER TABLE doses ADD PRIMARY KEY (IdDose)")
print('PK de doses criada.\n')

print('Criando PK de laboratorio')
cursor.execute("ALTER TABLE laboratorio ADD PRIMARY KEY (IdLaboratorio)")
print('PK de laboratorio criada.\n')

print('Criando PK de municipios')
cursor.execute("ALTER TABLE municipios ADD PRIMARY KEY (Código)")
print('PK de municipios criada.\n')

print('Crianda PK de pessoas')
cursor.execute("ALTER TABLE pessoas ADD PRIMARY KEY (Id)")
print('PK de pessoas criada.\n')

print('Crianda PK de unidade saude')
cursor.execute('ALTER TABLE `unidade saude` ADD PRIMARY KEY (IdUBS)')
print('PK de unidade saude criada.\n')

print('criando PK de vacinas')
cursor.execute('ALTER TABLE `vacinas` ADD PRIMARY KEY (IdVacina)')
print('PK de vacinas criada.\n')

Criando PK de doses
PK de doses criada.

Criando PK de laboratorio
PK de laboratorio criada.

Criando PK de municipios
PK de municipios criada.

Crianda PK de pessoas
PK de pessoas criada.

Crianda PK de unidade saude
PK de unidade saude criada.

criando PK de vacinas
PK de vacinas criada.



In [3]:
# Defigindo foreign key
cursor.execute('SET GLOBAL FOREIGN_KEY_CHECKS=0')

0

In [4]:
print('Criando FK s de enviada_para')
cursor.execute('ALTER TABLE enviada_para ADD FOREIGN KEY (IdVacina) REFERENCES vacinas(IdVacina)')
cursor.execute('ALTER TABLE enviada_para ADD FOREIGN KEY (IdUBS) REFERENCES `unidade saude`(IdUBS)')
print('FK s de enviada_para criadas.\n')

Criando FK s de enviada_para
FK s de enviada_para criadas.



In [5]:
print('Criando FK s de fica_no')
cursor.execute('ALTER TABLE fica_no ADD FOREIGN KEY (IdUBS) REFERENCES `unidade saude`(IdUBS)')
cursor.execute('ALTER TABLE fica_no ADD FOREIGN KEY (IdMunicipio) REFERENCES municipios(Código)')
print('FK s de fica_no criadas.\n')

Criando FK s de fica_no
FK s de fica_no criadas.



In [6]:
print('Criando FK s de habita_em')
cursor.execute('ALTER TABLE habita_em ADD FOREIGN KEY (IdPessoa) REFERENCES pessoas(Id)')
cursor.execute('ALTER TABLE habita_em ADD FOREIGN KEY (IdMunicipio) REFERENCES municipios(Código)')
print('FK s de habita_em criadas.\n')

Criando FK s de habita_em
FK s de habita_em criadas.



In [7]:
print('Criando FK s de produzido_por')
cursor.execute('ALTER TABLE produzida_por ADD FOREIGN KEY (IdVacina) REFERENCES vacinas(IdVacina)')
cursor.execute('ALTER TABLE produzida_por ADD FOREIGN KEY (IdLaboratorio) REFERENCES laboratorio(IdLaboratorio)')
print('FK s de produzido_por criadas.\n')

Criando FK s de produzido_por
FK s de produzido_por criadas.



In [ ]:
print('Criando FK s de tem')
cursor.execute('ALTER TABLE tem ADD FOREIGN KEY (IdDose) REFERENCES doses(IdDose)')
cursor.execute('ALTER TABLE tem ADD FOREIGN KEY (IdUBS) REFERENCES `unidade saude`(IdUBS)')
print('FK s de tem criadas.\n')

Criando FK s de tem


In [ ]:
print('Criando FK s de do_tipo')
cursor.execute('ALTER TABLE do_tipo ADD FOREIGN KEY (IdDose) REFERENCES doses(IdDose)')
cursor.execute('ALTER TABLE do_tipo ADD FOREIGN KEY (IdVacina) REFERENCES vacinas(IdVacina)')
print('FK s de do_tipo criadas.\n')

In [11]:
print('Criando FK s de aplicada_em')
cursor.execute('ALTER TABLE aplicada_em ADD FOREIGN KEY (IdPessoa) REFERENCES pessoas(Id)')
cursor.execute('ALTER TABLE aplicada_em ADD FOREIGN KEY (IdDose) REFERENCES doses(IdDose)')
print('FK s de aplicada_em criadas.\n\n')

0

In [ ]:
cursor.execute('SET GLOBAL FOREIGN_KEY_CHECKS=1')
print('\nFim da execução.')